In [1]:
from keras import models, layers
from keras import Input
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D,\
Add, Multiply, UpSampling2D, Dropout, concatenate
 
from keras.datasets import cifar100
from keras.utils import to_categorical
from keras import layers    
    
import os
import matplotlib.pyplot as plt
import numpy as np
import math
import cv2
import pandas as pd

Using TensorFlow backend.
C:\Users\Yoon\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Yoon\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Yoon\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Yoon\AppData\Roaming

In [2]:
IMG_HEIGHT = 192
IMG_WIDTH = 320
IMG_CHANNELS = 6

Data Load

In [3]:
test_list = pd.read_csv('data/adobe240fps/test_list.txt',header=None)
train_list = pd.read_csv('data/adobe240fps/train_list.txt',header=None)

In [ ]:
#data_length = len(os.walk("data/train/flow").__next__()[2])
data_length = 30
picNum= 0
for i in range(data_length):
    frame_length = len(os.walk("data/train/re30fps/"+train_list.iloc[i,0].split('.')[0]).__next__()[2])-1
    picNum += frame_length

x_load = np.zeros([picNum,IMG_HEIGHT, IMG_WIDTH, 6])
count= 0

for i in range(data_length):
    print("data%d"%i)
    temp = np.array([])
    frame_length = len(os.walk("data/train/re30fps/"+train_list.iloc[i,0].split('.')[0]).__next__()[2])-1
    for j in range(frame_length):
        if(j%10==0):
            print("frame%d"%j)
        temp1 = cv2.imread("data/train/re30fps/"+train_list.iloc[i,0].split('.')[0]+"/frame%d.jpg"%j)
        temp2 = cv2.imread("data/train/re30fps/"+train_list.iloc[i,0].split('.')[0]+"/frame%d.jpg"%(j+1))
        temp1 = np.array(temp1)
        temp2 = np.array(temp2)
        for k in range(len(temp1)):
            for l in range(temp1.shape[1]):
                for m in range(3):
                    x_load[count,k,l,m] = temp1[k,l,m]
                    x_load[count,k,l,m+3] = temp2[k,l,m]
        count += 1
        

data0
frame0
frame10
frame20
frame30
data1
frame0
frame10
frame20
frame30
frame40
frame50
data2
frame0
frame10
frame20
frame30
data3
frame0
frame10
frame20
frame30
frame40
frame50
data4
frame0
frame10
frame20
frame30
data5
frame0
frame10
frame20
frame30
frame40
frame50
frame60
frame70
frame80
frame90
frame100
data6
frame0
frame10
frame20
frame30
frame40
frame50
frame60
frame70
frame80
frame90
data7
frame0
frame10
frame20
frame30
frame40
frame50
frame60
frame70
frame80
frame90
frame100
data8
frame0
frame10
frame20
frame30
frame40
data9
frame0
frame10
frame20
frame30
frame40
frame50
frame60
frame70
frame80
data10
frame0
frame10
frame20
frame30
frame40
data11
frame0
frame10
frame20
frame30
frame40
frame50
frame60
frame70
frame80
frame90
data12
frame0
frame10
frame20
frame30
frame40
frame50
frame60
frame70
frame80
data13
frame0
frame10
frame20
frame30
frame40
frame50
frame60
frame70
frame80
frame90
frame100
data14
frame0
frame10
frame20
frame30
frame40
frame50
frame60
frame70
frame80
data1

In [ ]:
x_load.shape

In [ ]:
y_train = pd.DataFrame()
for i in range(data_length):
    print("data%d"%i)
    temp = pd.read_csv('data/train/flow/'+train_list.iloc[i,0].split('.')[0]+".csv",header=None)
    y_train = y_train.append(temp, ignore_index=True)

y_train.shape

In [ ]:
y_train = np.array(y_train)
y_train = np.reshape(y_train,((count,IMG_HEIGHT,IMG_WIDTH,4)))
y_train.shape

In [34]:
y_load = np.zeros([picNum,IMG_HEIGHT, IMG_WIDTH, 3])
y_count = 0

for i in range(data_length):
    print("data%d"%i)
    frame_length = int(len(os.walk("data/train/resize/"+train_list.iloc[i,0].split('.')[0]).__next__()[2])/8)-1
    for j in range(frame_length):
        if(j%10==0):
            print("frame%d"%j)
        temp = cv2.imread("data/train/resize/"+train_list.iloc[i,0].split('.')[0]+"/frame%d.jpg"%((j*8)+4))
        temp = np.array(temp)
        y_load[y_count] = temp
        y_count += 1

        #y_load = np.append(y_load, temp)

data0
frame0
frame10
frame20
frame30
data1
frame0
frame10
frame20
frame30
frame40
frame50
data2
frame0
frame10
frame20
frame30
data3
frame0
frame10
frame20
frame30
frame40
frame50
data4
frame0
frame10
frame20
frame30
data5
frame0
frame10
frame20
frame30
frame40
frame50
frame60
frame70
frame80
frame90
frame100
data6
frame0
frame10
frame20
frame30
frame40
frame50
frame60
frame70
frame80
frame90
data7
frame0
frame10
frame20
frame30
frame40
frame50
frame60
frame70
frame80
frame90
frame100
data8
frame0
frame10
frame20
frame30
frame40
data9
frame0
frame10
frame20
frame30
frame40
frame50
frame60
frame70
frame80
data10
frame0
frame10
frame20
frame30
frame40
data11
frame0
frame10
frame20
frame30
frame40
frame50
frame60
frame70
frame80
frame90
data12
frame0
frame10
frame20
frame30
frame40
frame50
frame60
frame70
frame80
data13
frame0
frame10
frame20
frame30
frame40
frame50
frame60
frame70
frame80
frame90
frame100
data14
frame0
frame10
frame20
frame30
frame40
frame50
frame60
frame70
frame80
data1

In [37]:
y_load = np.reshape(y_load,(count,IMG_HEIGHT, IMG_WIDTH, -1))
y_load.shape

(2839, 192, 320, 3)

In [38]:
div = int(count * 0.8)

In [ ]:
train_x = np.array(x_load[:div,:])
train_x = train_x/255.
train_y = np.array(y_train[:div,:])


val_x = np.array(x_load[div:,:])
val_x = val_x/255.
val_y = np.array(y_train[div:,:])


In [45]:
train_x = np.array(x_load[:div,:])
train_x = train_x/255.
train_y = np.array(y_load[:div,:])
train_y = train_y/255.


val_x = np.array(x_load[div:,:])
val_x = val_x/255.
val_y = np.array(y_load[div:,:])
val_y = val_y/255.

In [ ]:
train_y = np.reshape(train_y,(div,-1))
val_y = np.reshape(val_y,(count-div,-1))
print(train_y.shape)
print(val_y.shape)

In [40]:
train_y.shape

(2271, 192, 320, 3)

In [41]:
train_x.shape

(2271, 192, 320, 6)

Unet

In [ ]:
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

conv1 = Conv2D(32, 7, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
conv1 = Conv2D(32, 7, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
pool1 = MaxPooling2D(pool_size=(2,2))(conv1)

conv2 = Conv2D(64, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = Conv2D(64, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
pool2 = MaxPooling2D(pool_size=(2,2))(conv2)

conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
pool3 = MaxPooling2D(pool_size=(2,2))(conv3)

conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = Dropout(0.5)(conv4) 
pool4 = MaxPooling2D(pool_size=(2,2))(drop4)

conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = Dropout(0.5)(conv5)

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size=(2,2))(drop5))
merge6 = concatenate([drop4,up6], axis=3)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
merge7 = concatenate([conv3,up7], axis = 3)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
merge8 = concatenate([conv2,up8], axis = 3)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
merge9 = concatenate([conv1,up9], axis = 3)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
#conv9 = Conv2D(4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)

#conv10 = Conv2D(4, 1, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv9)

fc1 = GlobalAveragePooling2D()(conv9)
dense1 = Dense(IMG_HEIGHT*IMG_WIDTH*3)(fc1)

model = Model(inputs, dense1)
model.summary()

Unet2

In [ ]:
inputs = Input((IMG_HEIGHT, IMG_WIDTH, 16))

conv1 = Conv2D(32, 7, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
conv1 = Conv2D(32, 7, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
pool1 = MaxPooling2D(pool_size=(2,2))(conv1)

conv2 = Conv2D(64, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = Conv2D(64, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
pool2 = MaxPooling2D(pool_size=(2,2))(conv2)

conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
pool3 = MaxPooling2D(pool_size=(2,2))(conv3)

conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = Dropout(0.5)(conv4) 
pool4 = MaxPooling2D(pool_size=(2,2))(drop4)

conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = Dropout(0.5)(conv5)

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size=(2,2))(drop5))
merge6 = concatenate([drop4,up6], axis=3)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
merge7 = concatenate([conv3,up7], axis = 3)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
merge8 = concatenate([conv2,up8], axis = 3)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
merge9 = concatenate([conv1,up9], axis = 3)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv9 = Conv2D(12, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv9)

model = Model(inputs, conv9)
model.summary()

Unet3

In [70]:
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

conv1 = Conv2D(32, 7, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
conv1 = Conv2D(32, 7, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
pool1 = MaxPooling2D(pool_size=(2,2))(conv1)

conv2 = Conv2D(64, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = Conv2D(64, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
pool2 = MaxPooling2D(pool_size=(2,2))(conv2)

conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
pool3 = MaxPooling2D(pool_size=(2,2))(conv3)

conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = Dropout(0.5)(conv4) 
pool4 = MaxPooling2D(pool_size=(2,2))(drop4)

conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = Dropout(0.5)(conv5)

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size=(2,2))(drop5))
merge6 = concatenate([drop4,up6], axis=3)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
merge7 = concatenate([conv3,up7], axis = 3)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
merge8 = concatenate([conv2,up8], axis = 3)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
merge9 = concatenate([conv1,up9], axis = 3)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)

fc1 = GlobalAveragePooling2D()(conv9)

#conv9 = Conv2D(3, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)

#conv10 = Conv2D(3, 1, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)

dense1 = Dense(IMG_HEIGHT*IMG_WIDTH*3)(fc1)

model = Model(inputs, dense1)
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 192, 320, 6)  0                                            
__________________________________________________________________________________________________
conv2d_115 (Conv2D)             (None, 192, 320, 32) 9440        input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_116 (Conv2D)             (None, 192, 320, 32) 50208       conv2d_115[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_21 (MaxPooling2D) (None, 96, 160, 32)  0           conv2d_116[0][0]                 
____________________________________________________________________________________________

In [ ]:
model.compile(optimizer = 'adam', loss = 'mse', metrics = ['mse'])

In [9]:
model.load_weights('flowComp_6ch_RGBnorm.h5')

In [ ]:
early_stopping = EarlyStopping(patience=15, mode='auto', monitor='val_loss')
history = model.fit(train_x, train_y, validation_data=(val_x, val_y),epochs = 30, batch_size = 10, callbacks=[early_stopping])

In [ ]:
model.save_weights('flowComp_6ch_RGBnorm.h5')

In [ ]:
pred = model.predict(val_x)

In [ ]:
pred.shape

In [88]:
pred = np.reshape(pred, (count-div,IMG_HEIGHT, IMG_WIDTH, -1))
pred.shape

(568, 192, 320, 3)

In [91]:
test = pred[2]*255
cv2.imwrite('pred.jpg',test)

True

In [96]:
for i in range(pred.shape[0]):
    cv2.imwrite('res/pred%d.jpg'%i,pred[i]*255)

In [ ]:
img3 = cv2.imread("data/train/true/720p_240fps_2/frame0.jpg", cv2.IMREAD_COLOR)
img4 = np.zeros_like(img3)

for i in range(img3.shape[0]):
    for j in range(img3.shape[1]):
        #if(i+flow[i,j,1]<=img3.shape[0] and j+flow[i,j,0]<=img3.shape[1]):
            #img4[int(i+(flow[i,j,1]/2)),int(j+(flow[i,j,0]/2))] = img3[i,j]
        y = int((i - test[i,j,1]/2))
        x = int((j - test[i,j,0]/2))
        if y>=720:
            y=719
        if y<0:
            y=0
        if x>=1280:
            x=1279
        if x<0:
            x=0
        img4[i,j] = img3[y,x]
cv2.imwrite('backWarp_fhat1.jpg',img4)

In [14]:
fNum = 0
for k in range(457,568):
    print("%d / %d"%(k,568))
    fNum = k-457

    img3 = cv2.imread("test/GOPR9656/frame%d.jpg"%fNum, cv2.IMREAD_COLOR)
    img4 = np.zeros_like(img3)

    for i in range(img3.shape[0]):
        for j in range(img3.shape[1]):
            if(i+pred[fNum,i,j,1]<=img3.shape[0] and j+pred[fNum,i,j,0]<=img3.shape[1]):
                img4[int(i+(pred[fNum,i,j,1])),int(j+(pred[fNum,i,j,0]))] = img3[i,j]
    for i in range(img3.shape[0]):
        for j in range(img3.shape[1]):
            if(sum(img4[i,j])==0):
                img4[i,j] = img3[i,j]
    cv2.imwrite('test/GOPR9656/frame%d_0.jpg'%fNum,img4)

457 / 568
458 / 568
459 / 568
460 / 568
461 / 568
462 / 568
463 / 568
464 / 568
465 / 568
466 / 568
467 / 568
468 / 568
469 / 568
470 / 568
471 / 568
472 / 568
473 / 568
474 / 568
475 / 568
476 / 568
477 / 568
478 / 568
479 / 568
480 / 568
481 / 568
482 / 568
483 / 568
484 / 568
485 / 568
486 / 568
487 / 568
488 / 568
489 / 568
490 / 568
491 / 568
492 / 568
493 / 568
494 / 568
495 / 568
496 / 568
497 / 568
498 / 568
499 / 568
500 / 568
501 / 568
502 / 568
503 / 568
504 / 568
505 / 568
506 / 568
507 / 568
508 / 568
509 / 568
510 / 568
511 / 568
512 / 568
513 / 568
514 / 568
515 / 568
516 / 568
517 / 568
518 / 568
519 / 568
520 / 568
521 / 568
522 / 568
523 / 568
524 / 568
525 / 568
526 / 568
527 / 568
528 / 568
529 / 568
530 / 568
531 / 568
532 / 568
533 / 568
534 / 568
535 / 568
536 / 568
537 / 568
538 / 568
539 / 568
540 / 568
541 / 568
542 / 568
543 / 568
544 / 568
545 / 568
546 / 568
547 / 568
548 / 568
549 / 568
550 / 568
551 / 568
552 / 568
553 / 568
554 / 568
555 / 568
556 / 568
